In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/farec'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/farec/train.csv
/kaggle/input/farec/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/farec/train.csv")
print(df.head())
df = df.replace("correct", 1).replace("incorrect", 0)
df['pickup_time'] = pd.to_datetime(df['pickup_time'])
df['pickup_year'] = pd.DatetimeIndex(df['pickup_time']).year
df['pickup_month'] = pd.DatetimeIndex(df['pickup_time']).month
df['pickup_day'] = pd.DatetimeIndex(df['pickup_time']).day
df['pickup_hour'] = pd.DatetimeIndex(df['pickup_time']).hour
df['pickup_minute'] = pd.DatetimeIndex(df['pickup_time']).minute

df['drop_time'] = pd.to_datetime(df['drop_time'])
df['drop_year'] = pd.DatetimeIndex(df['drop_time']).year
df['drop_month'] = pd.DatetimeIndex(df['drop_time']).month
df['drop_day'] = pd.DatetimeIndex(df['drop_time']).day
df['drop_hour'] = pd.DatetimeIndex(df['drop_time']).hour
df['drop_minute'] = pd.DatetimeIndex(df['drop_time']).minute


df = df.drop("pickup_time", axis=1).drop("drop_time", axis=1).drop("tripid", axis=1)
df=df.fillna(df.mean())



      tripid  additional_fare  duration  meter_waiting  meter_waiting_fare  \
0  189123628             10.5     834.0           56.0              0.0000   
1  189125358             10.5     791.0           47.0              0.0000   
2  189125719             10.5    1087.0           80.0              0.0000   
3  189127273             10.5     598.0          271.0             15.6638   
4  189128020              NaN       NaN            NaN                 NaN   

   meter_waiting_till_pickup     pickup_time       drop_time  pick_lat  \
0                       64.0  11/1/2019 0:20  11/1/2019 0:34   6.86252   
1                      134.0  11/1/2019 0:56  11/1/2019 1:09   6.88589   
2                       61.0  11/1/2019 1:08  11/1/2019 1:26   6.90839   
3                       68.0  11/1/2019 2:27  11/1/2019 2:37   6.92570   
4                        NaN  11/1/2019 3:34  11/1/2019 3:51   6.87441   

   pick_lon  drop_lat  drop_lon    fare    label  
0   79.8993   6.90330   79.8783  27

In [3]:
x = df.drop("label", axis=1)
y = df.label
print (x)
print (y)


       additional_fare     duration  meter_waiting  meter_waiting_fare  \
0            10.500000   834.000000      56.000000            0.000000   
1            10.500000   791.000000      47.000000            0.000000   
2            10.500000  1087.000000      80.000000            0.000000   
3            10.500000   598.000000     271.000000           15.663800   
4            13.719651  1702.858077     629.074231           32.057666   
...                ...          ...            ...                 ...   
17171        10.500000   838.000000      93.000000            5.421900   
17172        10.500000  2151.000000     428.000000            0.000000   
17173        10.500000   263.000000       9.000000            0.000000   
17174        10.500000   858.000000     115.000000            0.000000   
17175        10.500000   262.000000      21.000000            1.224300   

       meter_waiting_till_pickup  pick_lat  pick_lon  drop_lat  drop_lon  \
0                      64.000000   

In [15]:
# evaluate gradient boosting ensemble for regression
from numpy import mean
from numpy import std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingClassifier

# define the model
model = GradientBoostingClassifier()
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -0.061 (0.006)


In [16]:
df2 = pd.read_csv("/kaggle/input/farec/test.csv")

tripid = df2.tripid
df2['pickup_time'] = pd.to_datetime(df2['pickup_time'])
df2['pickup_year'] = pd.DatetimeIndex(df2['pickup_time']).year
df2['pickup_month'] = pd.DatetimeIndex(df2['pickup_time']).month
df2['pickup_day'] = pd.DatetimeIndex(df2['pickup_time']).day
df2['pickup_hour'] = pd.DatetimeIndex(df2['pickup_time']).hour
df2['pickup_minute'] = pd.DatetimeIndex(df2['pickup_time']).minute

df2['drop_time'] = pd.to_datetime(df2['drop_time'])
df2['drop_year'] = pd.DatetimeIndex(df2['drop_time']).year
df2['drop_month'] = pd.DatetimeIndex(df2['drop_time']).month
df2['drop_day'] = pd.DatetimeIndex(df2['drop_time']).day
df2['drop_hour'] = pd.DatetimeIndex(df2['drop_time']).hour
df2['drop_minute'] = pd.DatetimeIndex(df2['drop_time']).minute

df2 = df2.drop("pickup_time", axis=1).drop("drop_time", axis=1).drop("tripid", axis=1)
df2.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,drop_year,drop_month,drop_day,drop_hour,drop_minute
0,10.5,924,42,2.4486,148,6.83454,79.8750,6.77490,79.8840,289.27,2020,2,1,0,38,2020,2,1,0,53
1,10.5,4249,20,0.0000,91,6.91168,79.8723,6.55091,79.9706,1912.70,2020,2,1,1,2,2020,2,1,2,13
2,10.5,1552,255,2.6588,23,6.92145,79.8478,6.90539,79.8989,394.00,2020,2,1,5,2,2020,2,1,5,28
3,10.5,462,16,0.0000,198,6.77433,79.9416,6.80401,79.9407,154.32,2020,2,1,5,30,2020,2,1,5,38
4,10.5,814,392,12.3692,69,6.97968,79.9130,6.98875,79.8914,147.47,2020,2,1,7,0,2020,2,1,7,14


In [17]:
# define the model
model = GradientBoostingClassifier()
# fit the model on the whole dataset
model.fit(x, y)
# make a single prediction
p = model.predict(df2)
print(p)

[1 1 1 ... 1 1 1]


In [18]:
print(p[:20])

[1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1]


In [12]:




output = {'tripid' : tripid, 'prediction' : p}
result = pd.DataFrame(output)
print(result.head())

result.to_csv('submission.csv', index= False)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.